## Binary Classifier Evaluation
This notebook aims at assessing the performance of a binary classifier, calculating some useful evaluation metrics.

*Assumptions:*
- The predictions from the binary classifier are stored in the format adopted in the inference step implemented in `steps/infer.py`. This format expects the output classifier confidence score in a simple *numpy* array stored in an `npy` file, named after the image.
- Similarly, since at the end of this notebook CAMs will be visualized, these are assumed to be saved in a similar format. The CAM should be a *numpy* 2D array stored in a `npy` file, named after the image. This format is, again, the one adopted in the inference step implemented in `steps/infer.py`.
- Ground truth labels are assumed to be located in a JSON file describing a dictionary with a specific structure, described below. This is the same structure adopted by the [AerialWaste](https://aerialwaste.org) dataset.

*Assumed dataset structure:*
```python
{
   "categories":[{"id": 1, "name": "candidate_site"}],
   "images":[
      {
         "id":1,
         "categories":[1],
         "file_name":"1.png",
         "height":1000,
         "width":1000,
         ...
      },
      ...
   ]
}
```

In [ ]:
# Import libraries
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import (roc_curve, auc, precision_recall_curve, 
                             average_precision_score, confusion_matrix, 
                             f1_score, precision_score, accuracy_score, recall_score)
from tqdm.notebook import tqdm
from misc.utils import onehot, CAMVisualizer

In [ ]:
# Paths
ds_dir = "AerialWaste3.0"               # Dataset source directory
img_dir = os.path.join(ds_dir, "images")                # Directory with dataset images [needed for CAM visualization]
eval_json = os.path.join(ds_dir, "testing-binary.json")     # Path to JSON file with GT information of images to include in evaluation
class_out_dir = "test-experiment"   # Path to directory associated to the binary classifier experiment [defined for convenience]
pred_dir = os.path.join(class_out_dir, "predictions")       # Path to directory with the classifier output predictions
cams_dir = os.path.join(class_out_dir, "cams")              # Path to directory with the classifier output CAMs
# Check file/directory existence
assert os.path.isdir(img_dir)
assert os.path.isfile(eval_json)
assert os.path.isdir(pred_dir)
assert os.path.isdir(cams_dir)

In [ ]:
# Parameters
# Classification threshold
threshold_type = 'custom'   # 'custom' or 'best_f1'. Use the former to set a user-defined threshold, the latter to dynamically select the threshold yielding the best F1 score on the current set.
threshold_value = .5        # User-defined threshold value. Ignored if 'threshold_type' is 'best_f1'.

### Load ground truth
Load to a dedicated variable the content of the JSON file with ground truth information over the dataset to use for evaluation.

In [ ]:
with open(eval_json) as file:
    gt_content = json.load(file)

### Categories
Store information about target category from the dataset. This will be used to support execution of some of the following cells.

In [ ]:
categories = gt_content["categories"]               # Store category information in a dedicated variable for convenience
num_cats = len(categories)                          # Number of categories
cat_names = [cat["name"] for cat in categories]     # Category names
cat_ids = [cat["id"] for cat in categories]         # Category IDs
# Check number of categories is 1 --> verifies used dataset is suitable for binary classification
assert num_cats == 1 

### Load Predictions
Load the predictions for each image. Predictions will be stored in a dictionary with as keys the names of the image files, and as values the classification scores. Such a dictionary is exemplified below.
```python
{
   "1.png": 0.996947705745697,
   "2.png": 0.012907988391816616,
   "3.png": 0.5127884149551392,
    ...
}
```

In [ ]:
# Initialize empty dictionary to store predictions
predictions = dict()
# For each image, save necessary information in the newly-created dictionary
for image in tqdm(gt_content["images"], leave=False):
    fn = image["file_name"]             # Store image file name for convenience
    # Search and load associated prediction in dedicated folder
    pred_file = os.path.join(pred_dir, '.'.join(fn.split('.')[:-1])+'.npy')     # Handles also the case where '.' is found in the file name, before extension
    assert os.path.isfile(pred_file)    # Check file existence
    predictions[fn] = np.load(pred_file)[0]

### Compute `y_score`, `y_true` 
Extract information from predictions and ground truth to compose 2 arrays which will be crucial to compute all the metrics described in the following sections:
* `y_score`: classification scores output from the model. 1D array of length `num_images`.
* `y_true`: ground truth target values. 1D array of length `num_images`.

In [ ]:
y_score = np.array(list(predictions.values()))
y_true = np.array([onehot(image["categories"], cat_ids) for image in gt_content["images"]])

## Binary Classification Metrics
In the context of a binary classification task, the terms *positive* and *negative* are used to describe the classifier predictions, whereas the terms *true* and *false* refer to whether the prediction corresponds to the external judgment, i.e., the ground truth *observation*. Given these definitions, it is possible to formulate the following table:

<table>
   <tr>
      <th></th>
      <th colspan=2, style="text-align: center">Actual class (observation)</th>
   </tr>
   <tr>
      <th rowspan="2">Predicted class (expectation)</th>
      <td>True Positive (TP) Correct result</td>
      <td>False Positive (FP) Unexpected result</td>
   </tr>
   <tr>
      <td>False Negative (FN) Missing result</td>
      <td>True Negative (TN) Correct absence of result</td>
   </tr>
</table>

* **True Positives (TP)**: samples for which the model correctly predicted the positive class (e.g. the model inferred that a particular email message was spam, and it really was spam).
* **True Negatives (TN)**: samples for which which the model correctly predicted the negative class (e.g. the model inferred that a particular email message was not spam, and that email message really was not spam).
* **False Positives (FP)**: samples for which the model mistakenly predicted the positive class (e.g. the model inferred that a particular email message was spam (the positive class), but that email message was actually not spam).
* **False Negatives (FN)**: samples for which the model mistakenly predicted the negative class (e.g. the model inferred that a particular email message was not spam (the negative class), but that email message actually was spam).

### Classification score distribution
This section introduces the first plot to study the performance of a given model: a histogram describing the distribution of the classification scores output by the model across the whole range of possible output scores, i.e., the [0,1] range. To show this distribution, the histogram is composed by splitting the considered range into a set of bins, each collecting a portion of the classification scores. If the considered model were perfect and all its predictions were correct, the histogram would collect all negative samples in the first bin, i.e., the one closest to 0, and all positive samples in the last bin, i.e., the one closest to 1.  

In [ ]:
counts, bins = np.histogram(list(predictions.values()), bins=10)
plt.hist(bins[:-1], bins, weights=counts, edgecolor='black')
plt.xlim([0,1])
plt.xlabel('Classification score', fontsize=14)
plt.ylabel('Number of samples', fontsize=14)
plt.title('Classification score distribution', fontsize=18)
plt.show()

### ROC Curve and AUC
The **ROC curve** (Receiver Operating Characteristic curve) is a plot describing the performance of a classification model across all possible thresholds. To plot this curve, two specific metrics are needed:

* **True Positive Rate (TPR) / Recall**:

$$TPR = \frac{TP}{TP + FN}$$

* **False Positive Rate (FPR) / Fall-Out**: 

$$FPR = \frac{FP}{FP + TN}$$

Given these metrics, the ROC curve plots the TPR (y-axis) against the FPR (x-axis), after computing these values for all the available thresholds. The choice of this factor plays a crucial role, since lowering the classification threshold implies classifying more items as positive, thus increasing both False Positives and True Positives, which appear in the formulae reported above. Based on its ROC curve, a good classifier could be identified when the curve remains as far as possible from the diagonal toward the top-left corner, dashed in the following plot. As a final reference, the best model, i.e., the model always making correct predictions, should have its ROC passing from the (0,1) point and appearing as a horizontal line a the value 1.

The **AUC**, which stands for "Area Under the ROC Curve", is a metric strictly correlated to the ROC curve described above. This metric measures the area underneath the entire ROC curve in the rectangle with opposite vertices in (0,0) and (1,1). A possible iterpretation of this metric is the probability that the model ranks a random positive example higher than a random negative example.
AUC cam acquire values in the range between 0 and 1, with models making only wrong predictions having an AUC of 0, whereas models making 100% correct prediction having an AUC of 1.

AUC is a metric with 2 crucial properties, which make it suitable for comparing different models:
* **Threshold-invariance**: AUC provides a robust measure of the model performance by evaluating predictions across all possible thresholds;
* **Scale-invariance**: AUC evaluates predictions based on their ranking, rather than on their absolute values.

In [ ]:
fpr, tpr, thresholdsroc = roc_curve(y_true, y_score)
aucvalue = auc(fpr, tpr)
plt.plot(fpr, tpr, linewidth=2, label=None)
plt.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])                                
plt.title(f'ROC Curve, AUC: {aucvalue:.4f}', fontsize=18)    
plt.xlabel('False Positive Rate (Fall-Out)', fontsize=14)
plt.ylabel('True Positive Rate (Recall)', fontsize=14)
plt.grid(True)                            
plt.show()

### Precision-Recall curve and Average Precision
The ROC and AUC metrics described above are particularly useful when dealing with (roughly-)balanced datasets. In case the dataset is imbalanced, the Precision-Recall curve might be more effective. In order to plot this curve, two different metrics are needed:

* **Precision**: this metric is the ratio of the correctly predicted positives over all the samples predicted as positives. Therefore, this metric identifies the frequency with which the model was correct when predicting the positive class. It can also be seen as the classifier ability not to label as positive a sample that is truly negative.

$$P = \frac{TP}{TP + FP}$$

* **Recall**: it is the ratio of correctly predicted positive instances over the number of instances which are actually positive. It describes, out of all the possible correct positive labels, how many the model correctly identified. It can also be seen as the ability of the classifier to find all the positive samples.

$$R = \frac{TP}{TP + FN}$$

Precision and Recall raise the key issue of Precision-Recall tradeoff, which is also strictly related to the choice of the threshold value to consider a prediction positive. By raising the threshold, indeed, the number of positives diminishes, thus favoring Precision but lowering Recall. On the contrary, lowering the threshold implies raising the number of positive predictions, thus favoring Recall at the expenses of Precision

The Precision-Recall curve plots a comparison of these 2 metrics after collecting their value at different classification thresholds. In addition, the curve can also be summarized by another specific metric:

* **(Weighted) Average Precision**: this metric is a mean of the Precision values at each threshold, weighted by the increase in Recall from the previous threshold. Since the metric can be considered to resume the P-R curve, the metric can also be adopted to compare different models. Please, notice that sometimes the term AP refers to the simple mean of Precision values at all possible thresholds. In this notebook, the reference to AP always refers to its weighted counterpart, which considers both Precision and Recall.

$$AP = \frac{1}{n}\sum_{i=1}^{n} (R_i-R_{i-1})P_i$$

where *i* is the threshold index and *n* is the total number of considered thresholds.

In [ ]:
precs, recs, thresholds = precision_recall_curve(y_true, y_score)
ap = average_precision_score(y_true, y_score)

plt.plot(recs, precs, linewidth=2)
plt.ylim([0, 1])
plt.xlim([0, 1])
plt.title(f'Precision-Recall curve, AP: {ap:.4f}', fontsize=18)    
plt.xlabel('Recall', fontsize=14)
plt.ylabel('Precision', fontsize=14)
plt.grid(True)
plt.show();

As a useful support to the threshold study, the following cell plots the 3 main metrics at the various threshold values. For a description of F1-Score, see the cells below. 

In [ ]:
_, axes = plt.subplots(1,3, figsize=(24,6))
metrics = ['Precision', 'Recall', 'F1-Score']
for i, ax in enumerate(axes):
    match metrics[i]:
        case 'Precision':
            ax.plot(thresholds, precs[:-1], linewidth=2)
        case 'Recall':
            ax.plot(thresholds, recs[:-1], linewidth=2)
        case 'F1-Score':
            ax.plot(thresholds, (2*precs[:-1]*recs[:-1])/(precs[:-1]+recs[:-1]), linewidth=2)
    ax.set_ylim([0, 1])
    ax.set_xlim([0, 1])
    ax.set_title(f'{metrics[i]} by threshold', fontsize=18)    
    ax.set_xlabel('Thresholds', fontsize=14)
    ax.set_ylabel(metrics[i], fontsize=14)
    ax.grid(True)
plt.show();

### Classification threshold selection
All the metrics described and computed in the following sections require a specific threshold to be selected, in order to allow the computation of TP, TN, FP and FN by setting a cut-off confidence for classifying an image as positive. This section focuses on selecting such threshold.

This notebook supports 2 types of thresholds, for which a specific parameter must be adjusted in the first cells of the notebook itself. Based on the value of the `threshold_type` parameter, the adopted threshold could be:
- `custom`, a user-defined fixed threshold, whose value should be specified above in the variable `threshold_value`.
- `best_f1`, a dynamically defined value which maximizes the F1-Score on the dataset used for evaluation. For more information about the F1-Score, please, see the following sections.

In [ ]:
# Select threshold
if threshold_type == 'best_f1':
    # Compute F1 scores
    f1s = 2*precs*recs/(precs+recs)
    threshold = thresholds[np.argmax(f1s)]
elif threshold_type == 'custom':
    threshold = threshold_value
else:
    raise ValueError('Not supported threshold type passed as parameter.')
    
# Print summary of selected threshold
print(f"Classification threshold: {threshold:.4f}")

### Compute `y_pred`
Compute predictions by filtering scores on the just-selected. This process of another array, similar to `y_score` and `y_true` computed before: 
* `y_pred`: targets as predicted by the classifier. 1D array of length `num_images`.

In [ ]:
y_pred = (y_score > threshold)*1

### Confusion Matrix
The confusion matrix is a table to visualize the distribution of samples across the predicted and expected categories. The table has 2 dimensions ('Predicted' and 'Actual') with identical sets of classes, i.e., those considered in the specific problem. In a binary classification task, the classes are Positive (1) and Negative (0), thus allowing the creation of 4 cells at their intersection. In these cells lie the numbers of TP, TN, FP and FN.

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
# Plot confusion matrix
texts = [["TN", "FP"], ["FN", "TP"]]
ticks = [0, 1]
plt.title(f"Confusion Matrix", fontsize=18)
plt.xlabel("Predicted", fontsize=14)
plt.ylabel("Actual", fontsize=14)
plt.xticks(ticks, fontsize=12)
plt.yticks(ticks, fontsize=12)
thresh = cm.max() / 2
plt.imshow(cm, cmap=plt.cm.Blues, interpolation="nearest")
for i in range(2):
    for j in range(2):
        text = texts[i][j] + "\n\n" + str(cm[i, j])
        color = "white" if cm[i, j] > thresh else "black"
        alignment = "center"
        plt.text(j, i, text, 
                 horizontalalignment=alignment,
                 verticalalignment=alignment, 
                 color=color, fontsize=16)
plt.show()

### Accuracy, Precision, Recall and F1 

This section computes 4 of the most relevant metrics for any classification problem.

* **Accuracy**: fraction of correct predictions.

$$accuracy = \frac{TP + TN}{TP + TN + FP +FN}$$

* **Precision**: accuracy of positive predictions. Identifies the frequency with which a model was correct when predicting the positive class and can be seen as the ability of the classifier not to label as positive a sample that is negative.

$$precision = \frac{TP}{TP + FP}$$

* **Recall**: ratio of positive instances that are correctly detected by the classfier. It describes how many of all the actual positive samples in the dataset are correctly identified by model and can also be seen as the ability of the classifier to find all the positive samples.

$$recall = \frac{TP}{TP + FN}$$

* **F1 Score**: the harmonic mean of precision and recall, thus combining these 2 metrics into a single one.

$$F_1 = \frac{2}{\frac{1}{recall} + \frac{1}{precision}} = 2 * \frac{precision*recall}{precision+recall} = \frac{TP}{TP + \frac{FN + FP}{2}}$$

In [ ]:
# Compute metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
# Print metrics
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec*100:.2f}%")
print(f"Recall: {rec*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")
print(f"Average Precision (AP): {ap*100:.2f}%")

## Visualize CAMs

In [ ]:
# Separate images into True/False Positives/Negatives
tps = [img for img in gt_content["images"] if 1 in img["categories"] and (threshold < predictions[img["file_name"]])]
fps = [img for img in gt_content["images"] if 1 not in img["categories"] and (threshold < predictions[img["file_name"]])]
tns = [img for img in gt_content["images"] if 1 not in img["categories"] and (threshold > predictions[img["file_name"]])]
fns = [img for img in gt_content["images"] if 1 in img["categories"] and (threshold > predictions[img["file_name"]])]

In [ ]:
# Select images to see CAMs of
images = [img for img in gt_content["images"]]
image_paths = [os.path.join(img_dir, img["file_name"]) for img in images]

In [ ]:
# Extract list of category names (actually, list with a single cat)
cats_names = [gt_content["categories"][0]["name"]]

In [ ]:
visualizer = CAMVisualizer(image_paths, 
                           cat_names=cat_names, 
                           cams_dir=cams_dir,
                           preds_dir=pred_dir,
                           plot_cols=2)
visualizer.start()